In [67]:
import cv2
from google.colab.patches import cv2_imshow
import os 
import numpy as np 
import pandas as pd 
import math 
from PIL import Image 
from pathlib import Path
import PIL 
from keras.preprocessing import image 
from keras.models import Model,load_model 
from keras.applications.imagenet_utils import preprocess_input,decode_predictions 
from keras.applications.resnet50 import ResNet50
from matplotlib import pyplot as plt
from glob import glob
from google.colab import drive
from tqdm import tqdm
import os
import shutil
import random

In [68]:
 def model_cnn_load():
        channels=3
        img_dim=224
        model = ResNet50(weights = "imagenet", include_top=True,input_shape=(img_dim,img_dim,channels))
        out = model.layers[-2].output
        model_final = Model(inputs=model.input,outputs=out)
        return model_final


In [151]:
  def load_image(path):
        #print(path)
        img = cv2.imread(str(path))
        dimensions=img.shape
        height=img.shape[0]
        width=img.shape[1]
        img_dim=224
        img = cv2.resize(img,(img_dim,img_dim))
        return img 

In [143]:
# def video_to_frames(video_path,v_dir):
        
 #   cap = cv2.VideoCapture(video_path)
  #  frameRate = cap.get(5) #frame rate
   # print('frameRate ',frameRate)
   # while(cap.isOpened()):
   #     frameId = cap.get(1) #current frame number
        #print('frameId ',frameId)
    #    ret, frame = cap.read()
    #    if (ret != True):
     #       break
    #    if (frameId % math.floor(frameRate) == 0):
    #      frame_path=v_dir.joinpath(str(int(frameId))+'.jpg')
          #frame_path.touch()
    #      cv2.imwrite(str(frame_path), frame)
            
        #print(i)

   # cap.release()  
   # cv2.destroyAllWindows()

In [45]:
drive.mount('/content/drive')

Mounted at /content/drive


In [153]:

feat_dir=Path('/content/drive/My Drive/capstone_data/feat_dir')
frame_dir=Path('/content/drive/My Drive/capstone_data/temp_dir')
source_dir= Path('/content/drive/My Drive/capstone_data/YouTubeClips_Eng')
processed_files=[x.stem for x in feat_dir.glob('*')]
all_files=[x.stem for x in source_dir.glob('*')]
unprocess_files=[item for item in all_files if item not in processed_files]

In [154]:
print(len(processed_files))
print(len(all_files))
print(len(unprocess_files))


0
1617
1617


In [144]:
tem_dir=Path('/content/drive/My Drive/capstone_data/temp_dir/zYcY4mjLpxU_104_118')
if tem_dir:
  print(" cleanup: " + str(tem_dir) + "/")
  shutil.rmtree(tem_dir,ignore_errors=True)
        #os.mkdir(self.temp_dest)

 cleanup: /content/drive/My Drive/capstone_data/temp_dir/zYcY4mjLpxU_104_118/


In [155]:
def extract_frames(video_path, v_dir):
    cap = cv2.VideoCapture(video_path)
    frameRate = cap.get(5) #frame rate
    print('frameRate ',frameRate)
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        #print('frameId ',frameId)
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
          frame_path=v_dir.joinpath(str(int(frameId))+'.jpg')
          #frame_path.touch()
          if frame_path:
            #print(" cleanup: " + str(frame_path) + "/")
            shutil.rmtree(frame_path,ignore_errors=True)
          cv2.imwrite(str(frame_path), frame)
    cap.release()
    cv2.destroyAllWindows()

In [156]:
model=model_cnn_load()
def convert_vid_to_numpy(process_file):
  frames_step=80
  img_dim=224
  channels=3
  batch_size=20
  print('Processing video: '+str(item)+" - "+str(item.name) + " - "+ str(item.stem))
  videoName=item.stem
  v_dir=frame_dir.joinpath(item.stem)
  #print(v_dir)
  v_dir.mkdir(parents=True)
  extract_frames(str(item), v_dir)
  image_list=sorted(v_dir.glob('*.jpg')) #list of sorted key frames
  samples = np.round(np.linspace(0, len(image_list) - 1, frames_step)) #evenly spaced numbers
  image_list = [image_list[int(sample)] for sample in samples] #each sample of linspace
  images = np.zeros((len(image_list), img_dim, img_dim, channels)) # empty image array
  for i in range(len(image_list)):
    #print(image_list[i])
    img = load_image(image_list[i]) # load immage
    images[i] = img  #save images as list
  images = np.array(images)#convert to array of images
  fc_feats = model.predict(images,batch_size) # predict dense features from model
  img_feats = np.array(fc_feats) # save feature as array
  outfile = feat_dir.joinpath(item.stem+'.npy')
  np.save(str(outfile), img_feats) #save as file
  remove_dir(v_dir) # remove video directory as we got numpy

In [123]:

def remove_dir(frame_dir):
  shutil.rmtree(frame_dir, ignore_errors=True)

In [157]:
process_count=0
for item in tqdm(source_dir.glob('**/*.avi')):
  print(item)
  if(item.stem in unprocess_files):
    print(item)
    convert_vid_to_numpy(item)
    process_count=process_count+1
#shutil.rmtree(frame_dir, ignore_errors=True)
print('Total number of videos processed '+str(process_count))






0it [00:00, ?it/s]

/content/drive/My Drive/capstone_data/YouTubeClips_Eng/FCjpuJaUec0_19_26.avi
/content/drive/My Drive/capstone_data/YouTubeClips_Eng/FCjpuJaUec0_19_26.avi
Processing video: /content/drive/My Drive/capstone_data/YouTubeClips_Eng/FCjpuJaUec0_19_26.avi - FCjpuJaUec0_19_26.avi - FCjpuJaUec0_19_26
frameRate  30.0







1it [00:02,  2.01s/it]

/content/drive/My Drive/capstone_data/YouTubeClips_Eng/f-24IxG9ijw_25_40.avi
/content/drive/My Drive/capstone_data/YouTubeClips_Eng/f-24IxG9ijw_25_40.avi
Processing video: /content/drive/My Drive/capstone_data/YouTubeClips_Eng/f-24IxG9ijw_25_40.avi - f-24IxG9ijw_25_40.avi - f-24IxG9ijw_25_40
frameRate  25.0







2it [00:03,  1.92s/it]

...

/content/drive/My Drive/capstone_data/YouTubeClips_Eng/9LSuyLyuUiM_3_6.avi
/content/drive/My Drive/capstone_data/YouTubeClips_Eng/9LSuyLyuUiM_3_6.avi
Processing video: /content/drive/My Drive/capstone_data/YouTubeClips_Eng/9LSuyLyuUiM_3_6.avi - 9LSuyLyuUiM_3_6.avi - 9LSuyLyuUiM_3_6
frameRate  29.97002997002997







1616it [52:29,  1.38s/it]

/content/drive/My Drive/capstone_data/YouTubeClips_Eng/d7Gs0uGFLh0_5_13.avi
/content/drive/My Drive/capstone_data/YouTubeClips_Eng/d7Gs0uGFLh0_5_13.avi
Processing video: /content/drive/My Drive/capstone_data/YouTubeClips_Eng/d7Gs0uGFLh0_5_13.avi - d7Gs0uGFLh0_5_13.avi - d7Gs0uGFLh0_5_13
frameRate  59.916666666666664







1617it [52:30,  1.95s/it]

Total number of videos processed 1617


In [121]:
img_path='/content/drive/My Drive/capstone_data/temp_dir/zYcY4mjLpxU_104_118/0.jpg'
load_image(img_path)

/content/drive/My Drive/capstone_data/temp_dir/zYcY4mjLpxU_104_118/0.jpg


array([[[ 68,  58, 118],
        [ 69,  60, 116],
        [ 70,  62, 115],
        ...,
        [ 84,  95, 124],
        [ 86, 100, 136],
        [ 92, 101, 144]],

       [[ 69,  60, 117],
        [ 70,  61, 117],
        [ 72,  65, 116],
        ...,
        [ 84,  95, 126],
        [ 87, 100, 138],
        [ 90, 102, 144]],

       [[ 70,  61, 117],
        [ 71,  63, 116],
        [ 75,  67, 118],
        ...,
        [ 84,  97, 131],
        [ 84,  99, 139],
        [ 90, 101, 145]],

       ...,

       [[103,  99, 159],
        [103,  98, 159],
        [103,  98, 159],
        ...,
        [ 29,   6,   0],
        [ 24,   5,   0],
        [ 22,   4,   1]],

       [[103,  98, 159],
        [104,  99, 160],
        [102,  97, 158],
        ...,
        [ 40,  14,   2],
        [ 32,  10,   2],
        [ 27,   9,   2]],

       [[103,  98, 159],
        [103,  98, 159],
        [102,  97, 158],
        ...,
        [ 49,  21,   5],
        [ 38,  16,   4],
        [ 32,  15,   8]]

In [118]:
for item in source_dir.glob('**/*.avi'):
  print(item)

/content/drive/My Drive/capstone_data/zYcY4mjLpxU_104_118.avi
